In [ ]:

def sparse_diags(Dia,M):
    dia = Dia #{0:1 , 1:2 , 2:3}
    
    #creating a zeros matrix (size = input image size)
    DM = np.zeros([M,M])
    
    
    #this for loop is used to traverse the rows of
    #zeros matrix to append the values in the 
    #appropriate position 
    for i in range(M):
        
        #this forloop is used to get the
        #position and then to add the values 
        #in that particular position 
        for j in dia:
            
            #this if statement is used to 
            #avoid the index bound error
            #if the position give goes beyond 
            #position of matrix with respect to
            #the main diagonal position
            if i+j >= 0 and i+j < M:
                DM[i][i+j] = dia[j]
    return sparse.coo_matrix(DM)

def lsinpaintnew(I):
    
    p = np.where(I == 0) # creating tuple of arrays indicating the incides x equal to 0
    
    I[p] = np.nan # setting the element of x at indices in p to Nan(Not a number)
    
    he, wi = I.shape # assigining the no.of rows and columns w.r.t to x
    
    Dia = {0:1 , 1:-2 , 2:1} # creating dictionary that maps the key 0,1,2 to values 1,-2,1 .
    
    x1 = [] # creating an x1 empyt list to store  


    for i in range(he): # assigned to ith row of X
        
        y = I[i,:]  # assigned to ith row of X
    
        if np.sum(np.isnan(y)) > 0 :# assigned to length of y # checking if there are any Nan values in y
            
            N = len(y)  # assigned to length of y
            
            D = sparse_diags(Dia,N) # constructed using the dictionary and value of N
            
            k = np.isfinite(y) # indices where y is finite and k is boolean array and k logical vector, indexes known values               
            
            S = np.eye(N) # creating a identity matrix of size N
            
            S = S[k, :]  # selects the rows of S corresponding to true values in the boolean array k and  S  sampling matrix                      
            
            Sc = np.eye(N) # creating a identity matrix of size N    
            
            Sc = Sc[~k, :] # selects the roes of sc corresponding to False values in the boolean array k     
            
            v = -np.linalg.solve(Sc @ (D.T @ D) @ Sc.T, (Sc @ D.T @ D @ S.T @ y[k]))  #  v estimated samples 
            
            x = np.zeros((N,1))
            
            x[k] = y[k].reshape(len(k)-len(v),1)
            
            x[~k] = v.reshape(len(v),1)

        else:
            
            x = y
            
        x1.append(list(x))
        
    x2 = [] # creating an X2 empyt list to store 

    for j in range(wi): 
        
        y = np.array(list(map(lambda x:x[j],I)))
    
        if np.sum(np.isnan(y)) > 0: # assigned to length of y # checking if there are any Nan values in y
            
            N = len(y) # assigned to ith row of X

            D = sparse_diags(Dia,N)# constructed using the dictionary and value of N
            
            k = np.isfinite(y) # indices where y is finite and k is boolean array and k logical vector, indexes known values               
            
            S = np.eye(N) # creating a identity matrix of size N
            
            S = S[k, :]   # selects the rows of S corresponding to true values in the boolean array k and  S  sampling matrix                       
            
            Sc = np.eye(N) # creating a identity matrix of size N 
            
            Sc = Sc[~k, :] # selects the rows of sc corresponding to False values in the boolean array k and Sc  complement of S   
            
            v = -np.linalg.solve(Sc @ (D.T @ D) @ Sc.T, (Sc @ D.T @ D @ S.T @ y[k]))# v : estimated samples 
            
            x = np.zeros((N,1))
            
            x[k] = y[k].reshape(len(k)-len(v),1)
            
            x[~k] = v.reshape(len(v),1)

        else:
            x = y
            
        x2.append(list(x))
    
    
    return (np.array(x1,dtype=float)+(np.array(x2,dtype=float)).T)*0.5

image = io.imread('inpaint_img.tif').astype(np.float)

mask_image =Image.open('img_mask.tif')

restored_image = lsinpaintnew(image)

fig, ax = plt.subplots(1, 3, figsize=(15, 15))

ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original image')

ax[1].imshow(mask_image, cmap='gray')
ax[1].set_title('Mask image')


ax[2].imshow(restored_image, cmap='gray')
ax[2].set_title('Restored image')
